In [ ]:
import os
import v3io_frames as v3f
from v3io import dataplane
import requests
import urllib3
from mlrun import new_function
import yaml
with open("config.yaml") as f:
    config = yaml.safe_load(f)

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

DASHBOARD_URL = f"https://dashboard.{os.getenv('IGZ_NAMESPACE_DOMAIN')}"
session = requests.Session()
session.auth = (os.getenv("V3IO_USERNAME"), config['api']['auth'])
auth = session.post(f'{DASHBOARD_URL}/api/sessions', verify=False)

# Remove API Gateway
try:
    print(f"Deleting API Gateway '{config['api']['gateway']}'...")
    response = session.delete(f"{DASHBOARD_URL}/api/api_gateways/{config['api']['gateway']}")
except:
    print(f"Failed to delete API Gateway '{API_GATEWAY}'.")
    pass

# Remove functions
functions = [f"{config['project']['name']}-{config['api']['get_image_function']}", f"{config['project']['name']}-{config['api']['facial_recognition_function']}"]
for function in functions:
    try:
        print(f"Deleting Function '{function}'...")
        response = session.delete(f'{DASHBOARD_URL}/api/functions/{function}')
    except:
        print(f"Failed to delete Function '{function}'.")
        pass

# Remove streams
frames_client = v3f.Client("framesd:8081",container="bigdata")
streams = [config['stream']['raw_video_stream'], config['stream']['tagged_video_stream']]
for stream in streams:
    try:
        print(f"Deleting Stream '{stream}'...")
        frames_client.delete("stream",table=stream)
    except:
        print(f"Failed to delete Stream '{stream}'.")
        pass

# Remove table
v3io_client = v3f.Client('framesd:8081',container=config['project']['container'])
try:
    print(f"Deleting Table '{config['camera']['list_table']}'...")
    v3io_client.delete('kv', config['camera']['list_table'])
except:
    print(f"Table '{config['camera']['list_table']}' already deleted.")
    pass

print("Done.")